In [4]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


In [5]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [6]:
from langchain_core.messages import HumanMessage

# not persistence
model.invoke([
    HumanMessage("Hi, I am bob"),
])


AIMessage(content='Hello Bob! Nice to meet you.\n\nHow can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--a5ec6c82-1894-4ac1-9538-ad1df7e9b57d-0', usage_metadata={'input_tokens': 6, 'output_tokens': 485, 'total_tokens': 491, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 469}})

In [7]:
model.invoke([
    HumanMessage("What's my name?"),
])

AIMessage(content="As an AI, I don't have access to your personal information, so I don't know your name. You haven't told me yet!\n\nIf you'd like to tell me, I'll remember it for our current conversation. 😊", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f645deed-da47-4d40-8eb6-7fc741c8cab5-0', usage_metadata={'input_tokens': 7, 'output_tokens': 428, 'total_tokens': 435, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 375}})

In [8]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--220512db-04a4-4b84-abf0-edf66650fadf-0', usage_metadata={'input_tokens': 25, 'output_tokens': 26, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 21}})

# Message Persistence
LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.

LangGraph comes with a simple in-memory checkpointer, which we use below. See its documentation for more detail, including how to use different persistence backends (e.g., SQLite or Postgres).

In [9]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# workflow define
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the modal
def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {'messages': response}


workflow.add_edge(START, "model")
workflow.add_node('model', call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {'configurable': {"thread_id": "abc123"}}

In [12]:
query = "Hi, I'm Bob"
input_messages = [HumanMessage(content=query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello again, Bob! Yes, I definitely remember you're Bob. No need to introduce yourself again – I've got it locked in!

What can I help you with today?


In [18]:
memory.__dict__

{'serde': <langgraph.checkpoint.serde.jsonplus.JsonPlusSerializer at 0x7440402da6f0>,
 'storage': defaultdict(<function langgraph.checkpoint.memory.InMemorySaver.__init__.<locals>.<lambda>()>,
             {'abc123': defaultdict(dict,
                          {'': {'1f0862aa-dcb5-68fc-bfff-558dd27d9e10': (('msgpack',
                              b'\x86\xa1v\x04\xa2ts\xd9 2025-08-31T05:23:51.101867+00:00\xa2id\xd9$1f0862aa-dcb5-68fc-bfff-558dd27d9e10\xb0channel_versions\x81\xa9__start__\xd9400000000000000000000000000000001.0.29217707685063965\xadversions_seen\x81\xa9__input__\x80\xb0updated_channels\x91\xa9__start__'),
                             ('msgpack',
                              b'\x83\xa6source\xa5input\xa4step\xff\xa7parents\x80'),
                             None),
                            '1f0862aa-dcb7-6bca-8000-6783dcce4c80': (('msgpack',
                              b'\x86\xa1v\x04\xa2ts\xd9 2025-08-31T05:23:51.102757+00:00\xa2id\xd9$1f0862aa-dcb7-6bca-8000-6783d

In [19]:
query = "What's my name?"
input_messages = [HumanMessage(content=query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!


In [20]:
# change the thread id and test it.
config = {'configurable': {"thread_id": "abc234"}}
input_messages = [HumanMessage(content=query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. As an AI, I don't have access to personal information about you unless you tell me.


In [21]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!


# Use Prompt Template

In [22]:
# Prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You talk like a pirate. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
])

workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {'messages': response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [23]:
config = {'configurable': {"thread_id": "abc345"}}
query = "Hi, I'm Sifat"
input_messages = [HumanMessage(content=query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Sifat! A fine name for a landlubber, or even a salty dog like meself! Welcome aboard, matey! What brings ye to these here waters, eh?


In [24]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Shiver me timbers, ye just told me a moment ago! Your name be **Sifat**, if my memory ain't as leaky as a sieve! Did ye forget already, or be ye testin' an old pirate's wits? Har har!


In [26]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [27]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Sequence[BaseMessage]
    language: str

# new customize state
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [36]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output['messages'][-1].pretty_print(), output['language']

================================== Ai Message ==================================

¡Hola, Bob! Es un placer conocerte. ¿En qué puedo ayudarte hoy?


(None, 'Spanish')

In [37]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Como modelo de lenguaje, no tengo acceso a información personal sobre ti, incluyendo tu nombre. No sé quién eres.


# Managing List of Message & Message history.

In [39]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [40]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    print(f"Messages before trimming: {len(state['messages'])}")
    trimmed_messages = trimmer.invoke(state["messages"])
    print(f"Messages after trimming: {len(trimmed_messages)}")
    print("Remaining messages:")
    for msg in trimmed_messages:
        print(f"  {type(msg).__name__}: {msg.content}")
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [41]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

Messages before trimming: 12
Messages after trimming: 10
Remaining messages:
  SystemMessage: you're a good assistant
  HumanMessage: I like vanilla ice cream
  AIMessage: nice
  HumanMessage: whats 2 + 2
  AIMessage: 4
  HumanMessage: thanks
  AIMessage: no problem!
  HumanMessage: having fun?
  AIMessage: yes!
  HumanMessage: What is my name?
================================== Ai Message ==================================

I don't know your name. As an AI, I don't have access to personal information about you.


In [42]:
config = {"configurable": {"thread_id": "abc567"}}
query = "Do you know me?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

Messages before trimming: 12
Messages after trimming: 10
Remaining messages:
  SystemMessage: you're a good assistant
  HumanMessage: I like vanilla ice cream
  AIMessage: nice
  HumanMessage: whats 2 + 2
  AIMessage: 4
  HumanMessage: thanks
  AIMessage: no problem!
  HumanMessage: having fun?
  AIMessage: yes!
  HumanMessage: Do you know me?
================================== Ai Message ==================================

As an AI, I don't "know" you in the way a person knows another person. I don't have memory of past conversations or personal experiences with you.

My knowledge of you is limited to the current conversation we are having right now. I process the words you type and respond based on that.


In [43]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Messages before trimming: 1
Messages after trimming: 1
Remaining messages:
  HumanMessage: Hi I'm Todd, please tell me a joke.
Hi Todd! Here's one for you:

Why don't scientists trust atoms?

Because they make up everything!|